In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, ReLU
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense
from tensorflow.keras import Model

from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

import numpy as np

import timeit

In [10]:
# returns an ndarray
def preprocess(image):
    image = load_img(image, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    return image

# yhat is np.array((1,1000), dtype="float32")
def postprocess(yhat, outputname):
    yhat = yhat[outputname]
    # convert the probabilities to class labels
    label = decode_predictions(yhat, 3)
    for i in range(3):
        print(label[0][i])

# returns key1 in {key1: value1, key2: value2 ...}
def get_first_key(dictionary):
    for key in dictionary:
        return key
    return None

# returns a tuple (runner, inputname, outputname)
def get_runner_and_io_names(interpreter):
    # signature list is of the form {key: {inputs: value, outputs: values}}
    signature_dict = interpreter.get_signature_list()
    first_key = get_first_key(signature_dict)
    runner = interpreter.get_signature_runner(first_key)
    io_dict = signature_dict[first_key]
    inputname = io_dict['inputs'][0]
    outputname = io_dict['outputs'][0]
    return (runner, inputname, outputname)

# tflite based forward inference
def forward(image_array, runner, inputname):
    output = runner(input_1=image_array)
    return output

In [33]:
def quantize(array):
    s = 255 / (np.max(array) - np.min(array))
    f = lambda v: np.clip(np.round(s*v), -127, 127)
    quantized_array = f(array)
    return quantized_array

image_array = preprocess('images/mug.jpg')
get_scale(image_array)

1.0681031028120054

In [11]:
interpreter = tf.lite.Interpreter(model_path="vgg16-int8.tflite")
interpreter.allocate_tensors()

In [39]:
runner, inputname, outputname = get_runner_and_io_names(interpreter)
image_array = quantize(preprocess('images/mug.jpg')).astype(np.int8)#.astype(np.int8)
output = forward(image_array, runner, inputname)
postprocess(output, outputname)

('n03063599', 'coffee_mug', 57)
('n03063689', 'coffeepot', -101)
('n07930864', 'cup', -109)


In [14]:
np.array([0.5443, -12.333221, 3.65454, 3.82727, -0.456646, 257.62525]).astype(np.int8)

array([  0, -12,   3,   3,   0,   1], dtype=int8)

In [38]:
quantize(preprocess('images/mug.jpg')).astype(np.int8).dtype

dtype('int8')

In [32]:
print(np.min(preprocess('images/mug.jpg')),np.max(preprocess('images/mug.jpg')))

-123.68 115.061


In [ ]:
min max quant
array([[[[-109.,  -66.,  -59.],
         [-105.,  -66.,  -58.],
         [-108.,  -69.,  -62.],
         ...,
         [ -55.,  -33.,   -9.],
         [ -46.,  -28.,   -4.],
         [ -63.,  -47.,  -22.]],

        [[-108.,  -66.,  -59.],
         [-102.,  -66.,  -58.],
         [-107.,  -68.,  -61.],

casting
array([[[[-101,  -61,  -55],
         [ -97,  -61,  -54],
         [-100,  -64,  -57],
         ...,
         [ -51,  -30,   -8],
         [ -42,  -25,   -3],
         [ -58,  -43,  -20]],

        [[-100,  -61,  -55],
         [ -95,  -61,  -54],
         [ -99,  -63,  -56],